In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import librosa   #librairie qui traite les audios
import librosa.display 
from IPython.display import Audio 
import warnings 
warnings.filterwarnings('ignore')  #ignorer les warnings 



# Chargement des données

In [2]:
#Charger la dataset TESS 
paths  = []
labels = [] 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label = filename.split('_')[-1]
        print(label)
        label = label.split('.')[0]
        print(label)
        labels.append(label.lower())
    if len(paths) == 2800 :
        break
print('Chargement de la dataset terminé')
len(paths)

In [3]:
paths[:5]

In [4]:
labels[:5]

In [79]:
df = pd.DataFrame()
df['son'] = paths
df['label'] = labels 
df.head()
len(df)

# Exploration de données 

In [80]:
df['label'].value_counts()

In [8]:
sns.countplot(df['label'])

In [9]:
def waveplot(data, sr, emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion, size=20)
    librosa.display.waveshow(data,sr=sr)
    plt.show()

In [10]:
def spectogram(data, sr, emotion):
    x = librosa.stft(data)
    xdb = librosa.amplitude_to_db(abs(x))
    plt.figure(figsize=(10,4))
    plt.title(emotion, size=20)
    librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()

In [11]:
emotion = 'fear'
path = np.array(df['son'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [12]:
emotion = 'angry'
path = np.array(df['son'][df['label']==emotion])[1]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [13]:
emotion = 'disgust'
path = np.array(df['son'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [14]:
emotion = 'neutral'
path = np.array(df['son'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [15]:
emotion = 'sad'
path = np.array(df['son'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [16]:
emotion = 'ps'  #ps = pleasant surprise
path = np.array(df['son'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

In [17]:
emotion = 'happy'
path = np.array(df['son'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)

# Préparation des données  

In [18]:
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

In [19]:
extract_mfcc(df['son'][0])  

In [81]:
X_mfcc = df['son'].apply(lambda x: extract_mfcc(x))

In [82]:
X_mfcc

## Préparation pour le modèle LSTM

In [83]:
X = [x for x in X_mfcc]
X = np.array(X) 
X.shape

In [84]:
X = np.expand_dims(X, -1)
X.shape

In [87]:
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']]).toarray() 
y

In [88]:
y.shape

## Préparation pour le modèle CNN 

In [51]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
encoder = OneHotEncoder()
y = encoder.fit_transform(df[['label']]).toarray()
len(y)

In [52]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True)    
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [53]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [54]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# Modélisation 

## Modèle CNN 

In [55]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint

In [56]:
model_cnn=Sequential()
model_cnn.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(40, 1)))
model_cnn.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model_cnn.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model_cnn.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model_cnn.add(Dropout(0.2))

model_cnn.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model_cnn.add(Flatten())
model_cnn.add(Dense(units=32, activation='relu'))
model_cnn.add(Dropout(0.3))

model_cnn.add(Dense(units=7, activation='softmax'))
model_cnn.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model_cnn.summary()

In [58]:
from keras.callbacks import ReduceLROnPlateau

rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
history=model_cnn.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), callbacks=[rlrp])

In [61]:
print("Accuracy de notre modèle sur la test set : " , model_cnn.evaluate(x_test,y_test)[1]*100 , "%")

epochs = [i for i in range(50)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(epochs , train_loss , label = 'Loss du Training')
ax[0].plot(epochs , test_loss , label = 'Loss du Testing')
ax[0].set_title('Loss du Training & Testing')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs , train_acc , label = 'Accuracy du Training')      
ax[1].plot(epochs , test_acc , label = 'Accuracy du Testing')
ax[1].set_title('Accuracy du Training & Testing')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()

In [62]:
pred_test = model_cnn.predict(x_test)
y_pred = encoder.inverse_transform(pred_test)

y_test = encoder.inverse_transform(y_test)

In [63]:
df = pd.DataFrame(columns=['Labels prédits', 'Labels réels'])
df['Labels prédits'] = y_pred.flatten()
df['Labels réels'] = y_test.flatten()

df.head(10)

In [65]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize = (12, 10))
cm = pd.DataFrame(cm , index = [i for i in encoder.categories_] , columns = [i for i in encoder.categories_])
sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
plt.title('Matrice de confusion', size=20)
plt.xlabel('Labels prédits', size=14)
plt.ylabel('Labels réels', size=14)
plt.show()

In [66]:
print(classification_report(y_test, y_pred))

## Modèle LSTM 

In [89]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model_lstm = Sequential([
    LSTM(256, return_sequences=False, input_shape=(40,1)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

In [90]:
model_lstm = model_lstm.fit(X, y, validation_split=0.2, epochs=50, batch_size=64)   # 0.8 -> train set -> 2240 , 0.2 -> validation set  -> 560 


In [92]:
epochs = list(range(50))
acc = model_lstm.history['accuracy']
val_acc = model_lstm.history['val_accuracy']

plt.plot(epochs, acc, label='Accuracy du Training')
plt.plot(epochs, val_acc, label='Accuracy de la Validation')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [93]:
loss = model_lstm.history['loss']
val_loss = model_lstm.history['val_loss']

plt.plot(epochs, loss, label='Loss du Training')
plt.plot(epochs, val_loss, label='Loss de la Validation')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()